In [29]:
import pandas as pd
import random
from copy import deepcopy
from scipy.linalg import svd, diagsvd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
df = pd.read_csv('data/ratings_small.csv')
df.drop(columns=["timestamp"], inplace=True)
dfp = df.pivot(index=["userId"], columns=["movieId"])
dfp.fillna(2.5, inplace=True)
A = dfp.to_numpy()

In [31]:
def get(df, dfp):
    '''Retorna um index aleatório para (user X movie)'''
    ind = [i for i in dfp.index]
    col = [j for i,j in dfp.columns]
    i = df.iloc[random.randint(0,100004)]
    return (ind.index(int(i["userId"])), col.index(int(i["movieId"])))

In [32]:
def predict(df,dfp):
    A = dfp.to_numpy()
    B = deepcopy(A)

    ind = get(df,dfp)
    B[ind[0]][ind[1]] = random.randint(0,5)

    X, Y, Z = svd(B)
    Y[-650:] = 0
    sigma = diagsvd(Y, B.shape[0], B.shape[1])
    B = X @ sigma @ Z
    
    return ind, A[ind[0]][ind[1]], B[ind[0]][ind[1]]

In [33]:
# user, notaReal, notaCalculada = predict(df,dfp)
# print(user, notaReal, notaCalculada)
# print(abs(notaCalculada - notaReal))

In [34]:
def multPredictError(df,dfp,i):
    A = dfp.to_numpy()
    B = deepcopy(A)

    indices = []
    for _ in range(i):
        ind = get(df,dfp)
        B[ind[0]][ind[1]] = random.randint(0,5)
        indices.append(ind)

    X, Y, Z = svd(B)
    Y[-650:] = 0
    sigma = diagsvd(Y, B.shape[0], B.shape[1])
    B = X @ sigma @ Z

    error = np.array([abs(A[ind[0]][ind[1]] - B[ind[0]][ind[1]]) for ind in indices])
    return error.mean()

In [35]:
runs = 1000
error = [multPredictError(df,dfp,i) for i in range(1,runs)]
d = {"Quantidade de dados com ruído": list(range(1,runs)), "Média do Erro Absoluto": error}
print(d)

{'Quantidade de dados com ruído': [1, 2], 'Média do Erro Absoluto': [2.3578456833193235, 0.8684930601629064]}


In [36]:
dfa = pd.DataFrame.from_dict(d)
print(dfa)
dfa.to_csv('dados.csv', index=False)

   Quantidade de dados com ruído  Média do Erro Absoluto
0                              1                2.357846
1                              2                0.868493
